In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from core.yolov4 import YOLOv4, decode_train, compute_loss
from core.dataset import Dataset
from core.config import cfg
import numpy as np
from core import utils
from core.utils import freeze_all, unfreeze_all
import os
import shutil
import numpy as np

In [2]:
print("go")

go


In [3]:
def train_model(input_size, just_summary=True):
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    #------------------- dataset -----------
    trainset = Dataset(is_training=True, dataset_type="yolo")
    testset = Dataset(is_training=False, dataset_type="yolo")
    logdir = "./data/log"
    isfreeze = False
    steps_per_epoch =  len(trainset)
    first_stage_epochs = cfg.TRAIN.FISRT_STAGE_EPOCHS
    second_stage_epochs = cfg.TRAIN.SECOND_STAGE_EPOCHS
    global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
    warmup_steps = cfg.TRAIN.WARMUP_EPOCHS * steps_per_epoch
    total_steps = (first_stage_epochs + second_stage_epochs) * steps_per_epoch
    # --------------- end------------
    input_layer = layers.Input(shape=(cfg.TRAIN.INPUT_SIZE, cfg.TRAIN.INPUT_SIZE, 3))
    STRIDES, ANCHORS, NUM_CLASS, XYSCALE = utils.load_config()
    IOU_LOSS_THRESH = cfg.YOLO.IOU_LOSS_THRESH
    freeze_layers = utils.load_freeze_layer()
    feature_maps = YOLOv4(input_layer, NUM_CLASS)
    print("num classes:")
    print(NUM_CLASS)
#     ANCHORS = [12,16, 19,36, 40,28, 36,75, 76,55, 72,146, 142,110, 192,243, 459,401]
#     STRIDES = [8, 16, 32]
#     XYSCALE = [1.2, 1.1, 1.05]
    bbox_tensors = []
    for i, fm in enumerate(feature_maps):
        if i == 0:
            bbox_tensor = decode_train(fm, input_size // 8, NUM_CLASS, STRIDES, ANCHORS, i, XYSCALE)
        elif i == 1:
            bbox_tensor = decode_train(fm, input_size // 16, NUM_CLASS, STRIDES, ANCHORS, i, XYSCALE)
        else:
            bbox_tensor = decode_train(fm, input_size // 32, NUM_CLASS, STRIDES, ANCHORS, i, XYSCALE)
        bbox_tensors.append(fm)
        bbox_tensors.append(bbox_tensor)
            
    model = tf.keras.Model(input_layer, bbox_tensors)
    if just_summary:
        #print(model.summary())
        return model
    optimizer = tf.keras.optimizers.Adam()
    if os.path.exists(logdir): shutil.rmtree(logdir)
    writer = tf.summary.create_file_writer(logdir)

    # define training step function
    # @tf.function
    def train_step(image_data, target):
        with tf.GradientTape() as tape:
            pred_result = model(image_data, training=True)
            giou_loss = conf_loss = prob_loss = 0

            # optimizing process
            for i in range(len(freeze_layers)):
                conv, pred = pred_result[i * 2], pred_result[i * 2 + 1]
                loss_items = compute_loss(pred, conv, target[i][0], target[i][1], STRIDES=STRIDES, NUM_CLASS=NUM_CLASS, IOU_LOSS_THRESH=IOU_LOSS_THRESH, i=i)
                giou_loss += loss_items[0]
                conf_loss += loss_items[1]
                prob_loss += loss_items[2]

            total_loss = giou_loss + conf_loss + prob_loss

            gradients = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            tf.print("=> STEP %4d/%4d   lr: %.6f   giou_loss: %4.2f   conf_loss: %4.2f   "
                     "prob_loss: %4.2f   total_loss: %4.2f" % (global_steps, total_steps, optimizer.lr.numpy(),
                                                               giou_loss, conf_loss,
                                                               prob_loss, total_loss))
            # update learning rate
            global_steps.assign_add(1)
            if global_steps < warmup_steps:
                lr = global_steps / warmup_steps * cfg.TRAIN.LR_INIT
            else:
                lr = cfg.TRAIN.LR_END + 0.5 * (cfg.TRAIN.LR_INIT - cfg.TRAIN.LR_END) * (
                    (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi))
                )
            optimizer.lr.assign(lr.numpy())

            # writing summary data
            with writer.as_default():
                tf.summary.scalar("lr", optimizer.lr, step=global_steps)
                tf.summary.scalar("loss/total_loss", total_loss, step=global_steps)
                tf.summary.scalar("loss/giou_loss", giou_loss, step=global_steps)
                tf.summary.scalar("loss/conf_loss", conf_loss, step=global_steps)
                tf.summary.scalar("loss/prob_loss", prob_loss, step=global_steps)
            writer.flush()
    def test_step(image_data, target):
        with tf.GradientTape() as tape:
            pred_result = model(image_data, training=True)
            giou_loss = conf_loss = prob_loss = 0

            # optimizing process
            for i in range(len(freeze_layers)):
                conv, pred = pred_result[i * 2], pred_result[i * 2 + 1]
                loss_items = compute_loss(pred, conv, target[i][0], target[i][1], STRIDES=STRIDES, NUM_CLASS=NUM_CLASS, IOU_LOSS_THRESH=IOU_LOSS_THRESH, i=i)
                giou_loss += loss_items[0]
                conf_loss += loss_items[1]
                prob_loss += loss_items[2]

            total_loss = giou_loss + conf_loss + prob_loss

            tf.print("=> TEST STEP %4d   giou_loss: %4.2f   conf_loss: %4.2f   "
                     "prob_loss: %4.2f   total_loss: %4.2f" % (global_steps, giou_loss, conf_loss,
                                                               prob_loss, total_loss))

    print("Start training ....")
    # ----------------Training---- 
    for epoch in range(2):
        print(epoch)
        if epoch < first_stage_epochs:
            if not isfreeze:
                isfreeze = True
                for name in freeze_layers:
                    freeze = model.get_layer(name)
                    freeze_all(freeze)
        elif epoch >= first_stage_epochs:
            if isfreeze:
                isfreeze = False
                for name in freeze_layers:
                    freeze = model.get_layer(name)
                    unfreeze_all(freeze)
        for image_data, target in trainset:
            train_step(image_data, target)
        for image_data, target in testset:
            test_step(image_data, target)
        print(f"epoch {epoch} ends !!")
        model.save_weights("./checkpoints/yolov4")
        
    return model
    
    

In [4]:
model = train_model(416, False)

------------------------------
conv sahpe KerasTensor(type_spec=TensorSpec(shape=(None, 13, 13, 512), dtype=tf.float32, name=None), name='tf.nn.leaky_relu_3/LeakyRelu:0', description="created by layer 'tf.nn.leaky_relu_3'")
route_1 shape: KerasTensor(type_spec=TensorSpec(shape=(None, 52, 52, 256), dtype=tf.float32, name=None), name='tf.math.multiply_25/Mul:0', description="created by layer 'tf.math.multiply_25'")
num classes:
5
inside decoding
decode layer input before reshape: name: conv2d_66/BiasAdd:0 shape: (None, 52, 52, 30)
decode layer After reshape: name: tf.reshape/Reshape:0 shape: (None, 52, 52, 3, 10)
inside decoding
decode layer input before reshape: name: conv2d_74/BiasAdd:0 shape: (None, 26, 26, 30)
decode layer After reshape: name: tf.reshape_1/Reshape:0 shape: (None, 26, 26, 3, 10)
inside decoding
decode layer input before reshape: name: conv2d_82/BiasAdd:0 shape: (None, 13, 13, 30)
decode layer After reshape: name: tf.reshape_2/Reshape:0 shape: (None, 13, 13, 3, 10)
Sta

=> STEP   58/1750   lr: 0.000829   giou_loss: 8.98   conf_loss: 50.74   prob_loss: 11.96   total_loss: 71.68
=> STEP   59/1750   lr: 0.000843   giou_loss: 24.25   conf_loss: 66.72   prob_loss: 31.92   total_loss: 122.89
=> STEP   60/1750   lr: 0.000857   giou_loss: 3.66   conf_loss: 41.28   prob_loss: 4.58   total_loss: 49.53
=> STEP   61/1750   lr: 0.000871   giou_loss: 15.55   conf_loss: 59.88   prob_loss: 21.45   total_loss: 96.88
=> STEP   62/1750   lr: 0.000886   giou_loss: 2.10   conf_loss: 36.32   prob_loss: 3.31   total_loss: 41.73
=> STEP   63/1750   lr: 0.000900   giou_loss: 7.08   conf_loss: 39.93   prob_loss: 9.39   total_loss: 56.41
=> STEP   64/1750   lr: 0.000914   giou_loss: 12.58   conf_loss: 48.27   prob_loss: 11.80   total_loss: 72.65
=> STEP   65/1750   lr: 0.000929   giou_loss: 9.42   conf_loss: 39.96   prob_loss: 10.06   total_loss: 59.44
=> STEP   66/1750   lr: 0.000943   giou_loss: 6.87   conf_loss: 34.56   prob_loss: 11.57   total_loss: 52.99
=> STEP   67/1750 

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 416, 416, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 416, 416, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 416, 416, 32  128        ['conv2d[0][0]']                 
 rmalization)                   )                                                             

                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 208, 208, 64  0          ['tf.math.multiply_3[0][0]',     
 da)                            )                                 'tf.math.multiply_5[0][0]']     
                                                                                                  
 conv2d_6 (Conv2D)              (None, 208, 208, 64  4096        ['tf.__operators__.add[0][0]']   
                                )                                                                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 208, 208, 64  4096        ['tf.math.multiply_1[0][0]']     
                                )                                                                 
                                                                                                  
 batch_nor

 a)                             )                                 'tf.math.tanh_10[0][0]']        
                                                                                                  
 conv2d_11 (Conv2D)             (None, 104, 104, 64  4096        ['tf.math.multiply_10[0][0]']    
                                )                                                                 
                                                                                                  
 batch_normalization_12 (BatchN  (None, 104, 104, 64  256        ['conv2d_11[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 tf.math.softplus_11 (TFOpLambd  (None, 104, 104, 64  0          ['batch_normalization_12[0][0]'] 
 a)                             )                                                                 
          

 tf.math.softplus_9 (TFOpLambda  (None, 104, 104, 64  0          ['batch_normalization_10[0][0]'] 
 )                              )                                                                 
                                                                                                  
 tf.math.tanh_15 (TFOpLambda)   (None, 104, 104, 64  0           ['tf.math.softplus_15[0][0]']    
                                )                                                                 
                                                                                                  
 tf.math.tanh_9 (TFOpLambda)    (None, 104, 104, 64  0           ['tf.math.softplus_9[0][0]']     
                                )                                                                 
                                                                                                  
 tf.math.multiply_15 (TFOpLambd  (None, 104, 104, 64  0          ['batch_normalization_16[0][0]', 
 a)       

 tf.math.tanh_21 (TFOpLambda)   (None, 52, 52, 128)  0           ['tf.math.softplus_21[0][0]']    
                                                                                                  
 tf.math.multiply_21 (TFOpLambd  (None, 52, 52, 128)  0          ['batch_normalization_22[0][0]', 
 a)                                                               'tf.math.tanh_21[0][0]']        
                                                                                                  
 tf.__operators__.add_3 (TFOpLa  (None, 52, 52, 128)  0          ['tf.math.multiply_19[0][0]',    
 mbda)                                                            'tf.math.multiply_21[0][0]']    
                                                                                                  
 conv2d_22 (Conv2D)             (None, 52, 52, 128)  16384       ['tf.__operators__.add_3[0][0]'] 
                                                                                                  
 batch_nor

...

 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_11 (TFOpLambd  (None, 26, 26, 128)  0          ['batch_normalization_59[0][0]'] 
 a)                                                                                               
                                                                                                  
 tf.nn.leaky_relu_12 (TFOpLambd  (None, 52, 52, 128)  0          ['batch_normalization_60[0][0]'] 
 a)                                                                                               
                                                                                                  
 tf.image.resize_1 (TFOpLambda)  (None, 52, 52, 128)  0          ['tf.nn.leaky_relu_11[0][0]']    
                                                                                                  
 tf.concat

 conv2d_70 (Conv2D)             (None, 26, 26, 256)  131072      ['tf.nn.leaky_relu_21[0][0]']    
                                                                                                  
 batch_normalization_70 (BatchN  (None, 26, 26, 256)  1024       ['conv2d_70[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_22 (TFOpLambd  (None, 26, 26, 256)  0          ['batch_normalization_70[0][0]'] 
 a)                                                                                               
                                                                                                  
 conv2d_71 (Conv2D)             (None, 26, 26, 512)  1179648     ['tf.nn.leaky_relu_22[0][0]']    
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_81 (Conv2D)             (None, 13, 13, 1024  4718592     ['tf.nn.leaky_relu_31[0][0]']    
                                )                                                                 
                                                                                                  
 batch_normalization_66 (BatchN  (None, 52, 52, 256)  1024       ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_73 (BatchN  (None, 26, 26, 512)  2048       ['conv2d_73[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_nor

                                2),                                                               
                                 (None, 13, 13, 3,                                                
                                2),                                                               
                                 (None, 13, 13, 3,                                                
                                1),                                                               
                                 (None, 13, 13, 3,                                                
                                5)]                                                               
                                                                                                  
 tf.compat.v1.shape_5 (TFOpLamb  (5,)                0           ['tf.reshape_2[0][0]']           
 da)                                                                                              
          

 tf.math.multiply_48 (TFOpLambd  (None, 52, 52, 3, 2  0          ['tf.__operators__.add_11[0][0]']
 a)                             )                                                                 
                                                                                                  
 tf.math.multiply_49 (TFOpLambd  (None, 52, 52, 3, 2  0          ['tf.math.exp[0][0]']            
 a)                             )                                                                 
                                                                                                  
 tf.math.multiply_51 (TFOpLambd  (None, 26, 26, 3, 2  0          ['tf.__operators__.add_12[0][0]']
 a)                             )                                                                 
                                                                                                  
 tf.math.multiply_52 (TFOpLambd  (None, 26, 26, 3, 2  0          ['tf.math.exp_1[0][0]']          
 a)       

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 416, 416, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 416, 416, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 416, 416, 32  128        ['conv2d[0][0]']                 
 rmalization)                   )                                                             

                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 208, 208, 64  0          ['tf.math.multiply_3[0][0]',     
 da)                            )                                 'tf.math.multiply_5[0][0]']     
                                                                                                  
 conv2d_6 (Conv2D)              (None, 208, 208, 64  4096        ['tf.__operators__.add[0][0]']   
                                )                                                                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 208, 208, 64  4096        ['tf.math.multiply_1[0][0]']     
                                )                                                                 
                                                                                                  
 batch_nor

 a)                             )                                 'tf.math.tanh_10[0][0]']        
                                                                                                  
 conv2d_11 (Conv2D)             (None, 104, 104, 64  4096        ['tf.math.multiply_10[0][0]']    
                                )                                                                 
                                                                                                  
 batch_normalization_12 (BatchN  (None, 104, 104, 64  256        ['conv2d_11[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 tf.math.softplus_11 (TFOpLambd  (None, 104, 104, 64  0          ['batch_normalization_12[0][0]'] 
 a)                             )                                                                 
          

 tf.math.softplus_9 (TFOpLambda  (None, 104, 104, 64  0          ['batch_normalization_10[0][0]'] 
 )                              )                                                                 
                                                                                                  
 tf.math.tanh_15 (TFOpLambda)   (None, 104, 104, 64  0           ['tf.math.softplus_15[0][0]']    
                                )                                                                 
                                                                                                  
 tf.math.tanh_9 (TFOpLambda)    (None, 104, 104, 64  0           ['tf.math.softplus_9[0][0]']     
                                )                                                                 
                                                                                                  
 tf.math.multiply_15 (TFOpLambd  (None, 104, 104, 64  0          ['batch_normalization_16[0][0]', 
 a)       

 tf.math.tanh_21 (TFOpLambda)   (None, 52, 52, 128)  0           ['tf.math.softplus_21[0][0]']    
                                                                                                  
 tf.math.multiply_21 (TFOpLambd  (None, 52, 52, 128)  0          ['batch_normalization_22[0][0]', 
 a)                                                               'tf.math.tanh_21[0][0]']        
                                                                                                  
 tf.__operators__.add_3 (TFOpLa  (None, 52, 52, 128)  0          ['tf.math.multiply_19[0][0]',    
 mbda)                                                            'tf.math.multiply_21[0][0]']    
                                                                                                  
 conv2d_22 (Conv2D)             (None, 52, 52, 128)  16384       ['tf.__operators__.add_3[0][0]'] 
                                                                                                  
 batch_nor

...

 tf.concat_7 (TFOpLambda)       (None, 52, 52, 256)  0           ['tf.nn.leaky_relu_14[0][0]',    
                                                                  'tf.image.resize_1[0][0]']      
                                                                                                  
 conv2d_83 (Conv2D)             (None, 52, 52, 128)  32768       ['tf.concat_7[0][0]']            
                                                                                                  
 batch_normalization_84 (BatchN  (None, 52, 52, 128)  512        ['conv2d_83[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_15 (TFOpLambd  (None, 52, 52, 128)  0          ['batch_normalization_84[0][0]'] 
 a)                                                                                               
          

 batch_normalization_94 (BatchN  (None, 26, 26, 512)  2048       ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_25 (TFOpLambd  (None, 26, 26, 512)  0          ['batch_normalization_94[0][0]'] 
 a)                                                                                               
                                                                                                  
 conv2d_95 (Conv2D)             (None, 26, 26, 256)  131072      ['tf.nn.leaky_relu_25[0][0]']    
                                                                                                  
 batch_normalization_95 (BatchN  (None, 26, 26, 256)  1024       ['conv2d_95[0][0]']              
 ormalization)                                                                                    
          

 batch_normalization_103 (Batch  (None, 13, 13, 1024  4096       ['conv2d_104[0][0]']             
 Normalization)                 )                                                                 
                                                                                                  
 tf.nn.leaky_relu_20 (TFOpLambd  (None, 52, 52, 256)  0          ['batch_normalization_89[0][0]'] 
 a)                                                                                               
                                                                                                  
 tf.nn.leaky_relu_27 (TFOpLambd  (None, 26, 26, 512)  0          ['batch_normalization_96[0][0]'] 
 a)                                                                                               
                                                                                                  
 tf.nn.leaky_relu_34 (TFOpLambd  (None, 13, 13, 1024  0          ['batch_normalization_103[0][0]']
 a)       

                                                                                                  
 tf.math.sigmoid (TFOpLambda)   (None, 52, 52, 3, 2  0           ['tf.split[0][0]']               
                                )                                                                 
                                                                                                  
 tf.__operators__.getitem_1 (Sl  ()                  0           ['tf.compat.v1.shape_1[0][0]']   
 icingOpLambda)                                                                                   
                                                                                                  
 tf.math.sigmoid_3 (TFOpLambda)  (None, 26, 26, 3, 2  0          ['tf.split_1[0][0]']             
                                )                                                                 
                                                                                                  
 tf.__oper

 a)                             )                                                                 
                                                                                                  
 tf.math.multiply_75 (TFOpLambd  (None, 13, 13, 3, 2  0          ['tf.__operators__.add_23[0][0]']
 a)                             )                                                                 
                                                                                                  
 tf.math.multiply_76 (TFOpLambd  (None, 13, 13, 3, 2  0          ['tf.math.exp_2[0][0]']          
 a)                             )                                                                 
                                                                                                  
 tf.concat_10 (TFOpLambda)      (None, 52, 52, 3, 4  0           ['tf.math.multiply_69[0][0]',    
                                )                                 'tf.math.multiply_70[0][0]']    
          